# 1. All that we need
In this section we import the modules & packages required to achieve tasks in the code below.

In [1]:
from bs4 import BeautifulSoup
from difflib import SequenceMatcher
import math
import pandas as pd
from urllib import request
!pip install selenium
import time as time
from time import sleep
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException

# 2. Websites data

- Here the idea is to compact each website data in a list, and then to concatenate those lists in a list of lists (i.e.) an array) named __websites_data__, which will be the input of a function that will scrap all sites at once.
- __Each list has the following format__ :
    - URL, 
    - class name or css selector of match boxes, 
    - class name or css selector of contestants boxes, 
    - class name or css selector of contestants box (if there is only one box), 
    - class name or css selector of odd boxes, 
    - class name or css selector of match time, 
    - name of website, 
    - dummy variable = 0 or 1, 
    - if dummy variable is 1, xpath of the roller shutter button at the bottom of the page

In [2]:
winamax = ['https://www.winamax.fr/paris-sportifs/sports/1', '.sc-oTaAA.jpLhXr', '.sc-qWSGh', '.sc-prPLn', '.sc-fznWqX.sc-fzoiQi.sc-fzqARJ.sc-fznxKY.eBurPj', '.sc-qQlyE.jlXdeO', 'Winamax', 0]

france_pari_ligue1 = ['https://www.france-pari.fr/competition/96-parier-sur-ligue-1-ubereats', 'div.odd-event.uk-flex', 'span.bet-libEvent', 'span.bet-libEvent', 'span.odd', 'span.bet-time', 'France pari', 0]
france_pari_premier_league = ['https://www.france-pari.fr/competition/94-parier-sur-premier-league', 'div.odd-event.uk-flex', 'span.bet-libEvent', 'span.bet-libEvent', 'span.odd', 'span.bet-time', 'France pari', 0]
france_pari_laliga = ['https://www.france-pari.fr/competition/306-parier-sur-laliga', 'div.odd-event.uk-flex', 'span.bet-libEvent', 'span.bet-libEvent', 'span.odd', 'span.bet-time', 'France pari', 0]
france_pari_serie_a = ['https://www.france-pari.fr/competition/305-parier-sur-serie-a', 'div.odd-event.uk-flex', 'span.bet-libEvent', 'span.bet-libEvent', 'span.odd', 'span.bet-time', 'France pari', 0]
france_pari_bundesliga = ['https://www.france-pari.fr/competition/268-parier-sur-bundesliga', 'div.odd-event.uk-flex', 'span.bet-libEvent', 'span.bet-libEvent', 'span.odd', 'span.bet-time', 'France pari', 0]
france_pari_homepage = ['https://www.france-pari.fr/sport/13-parier-sur-football', 'div.odd-event.uk-flex', 'span.bet-libEvent', 'span.bet-libEvent', 'span.odd', 'span.bet-time', 'France pari', 0]
# France pari football section was split into different pages, 
# which explains why there are different lists for this website

parions_sport = ['https://www.enligne.parionssport.fdj.fr/paris-football', 'a.wpsel-bloc.wpsel-bloc__main', 'p.wpsel-desc', 'p.wpsel-desc', 'button.outcomeButton', 'span.wpsel-timerLabel', 'Parions sport', 0]

betclic = ['https://www.betclic.fr/football-s1', 'div.betBox_info', 'span.betBox_contestantName', 'span.betBox_contestantName', 'span.oddValue', 'span.betBox_matchDateInfo', 'Betclic', 0]

feelingbet = ['https://feelingbet.fr/sport/13-parier-sur-football', 'div.odd-event.uk-flex', 'span.bet-libEvent', 'span.bet-libEvent', 'span.odd', 'span.bet-time', 'Feelingbet', 0]

unibet = ['https://www.unibet.fr/sport/football', 'div.ui-touchlink.had-market.inline-market.calendar-event.cell', 'div.cell-event', 'div.cell-event', 'span.ui-touchlink-needsclick.price.odd-price', 'span.datetime', 'Unibet', 0]

zebet = ['https://www.zebet.fr/fr/sport/13-football', 'div.item-bloc.item-bloc-type-1', 'a.uk-text-truncate', 'a.uk-text-truncate', 'span.pmq-cote.uk-width-1-1', 'div.bet-time.uk-flex-item-none.uk-margin-left', 'Zebet', 0]

netbet = ['https://www.netbet.fr/football?tab=a-venir', 'div.nb-event.comingevents', 'div.nb-match_actor', 'div.nb-match_actor', 'div.nb-odds_amount', 'div.nb-event_timestart', 'Netbet', 1, '/html/body/div[1]/div/section/div/div/div[3]/ul[2]/li[3]/div/div[2]/button']


websites_data = [netbet, zebet, winamax, france_pari_ligue1, france_pari_premier_league, france_pari_laliga, france_pari_serie_a, france_pari_bundesliga, france_pari_homepage, parions_sport, feelingbet, unibet, betclic]
# Finally we aggregate the data

# 3. Data check
Since the Dataframes generator below requires quite some time to run, checking the existence of selected objects was a necessity (sometimes there are different ways to call a visual object with selectors, some of them often do not work and return an empty list).

In [4]:
def data_check(i):
    # i is the index of the website we want to check in websites data
    
    browser = webdriver.Firefox()
    # we choose the browser Selenium (or more exactly Geckodriver) will use as a marionnette
    # I prefer firefox, which is my usual browser
    
    browser.get(websites_data[i][0])
    time.sleep(2)
    # the browser opens the URL and takes some time off to let the Javascript execute itself
    
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # from the marionnette, ie the window seen by the user, we ask the remotely controlled driver to 
    # extract the html. It is the only way to scrap, since on betting sites all relevant data
    # are hidden (type = hidden is what appears on the page html)
    
    for k in range(1,6):
        if soup.select(websites_data[i][k]) != []:
            print('OK')
            print(len(soup.select(websites_data[i][k])))
        else:
            print('problem with coordinate', k+1)
    browser.close()
    # for all objects in a particular website list of coordinates, we see if Beautiful Soup
    # finds any objects in the html of the window under such designation.

In [5]:
data_check(9)
# for example, here we check the validity and amount of each type of data on "Parions sport" web page

OK
50
OK
98
OK
98
OK
294
OK
98


# 4. BS4 scrap function
*Given* :
- the __html__ of the window seen by the user (or here the Geckodriver, actually) 
- a list __data__ (which will become a list of dictionaries and henceforth be easily converted into a Pandas DataFrame)
- and __coordinates__, ie the class names or css selectors of the elements on the site

we scrap the objects we want (*contestants names, odds, time*) in every match box found by the driver.

In [6]:
def match_data(html, data, coordinates):
    
    soup = BeautifulSoup(html, 'html.parser')
    # BS4 translates html into a structure similar to the website

    matches = soup.select(coordinates[1])
    # Here I must confess I tried things to understand soup.select method
    # which appears to work indifferently with css_selector and class_name of a web element
    # therefore here we select match boxes: the returned object is a list!
    
    for match in matches:
    # ie "for each match box", which contains visually all other elements, and so in the html it 
    # will be the parent node of contestants, odds and time.
        
        curr_match_data = {}
        # we are going to build a dictionary for this particular match

        teams = match.select(coordinates[2])
        # teams is the selection of the "rectangle" which contains the teams
        # again, the finding of css selectors or class names is really visual
        # that is why different cases may occur
        
        if len(teams) >= 2:
        # if teams if a list which contains each contestant as a coordinate
        
            curr_match_data['Team 1'] = teams[0].text 
            curr_match_data['Team 2'] = teams[1].text 
            
        else:
        # if both contestants names are mingled in one string
        # below we are going to split the string
        
            teams = match.select(coordinates[3])
            # it is a different selection because it happens: on a same page there are two kinds of teams structures
            # very often the difference appears in the distinct displays of Live and later events
            
            if len(teams):
            # just to prevent from the case teams is an empty list!
            
                if len(teams[0].text.split(' - ')) >=2 :
                    curr_match_data['Team 1'] = teams[0].text.split(' - ')[0]
                    curr_match_data['Team 2'] = teams[0].text.split(' - ')[1]
                # here and below we split by a separator, sometimes it is '-', sometimes '/', sometimes '//'
                # not very exciting, I must admit...
                    
                elif len(teams[0].text.split(' / ')) >=2:
                    curr_match_data['Team 1'] = teams[0].text.split(' / ')[0]
                    curr_match_data['Team 2'] = teams[0].text.split(' / ')[1]
                elif len(teams[0].text.split(' // ')) >=2:
                    curr_match_data['Team 1'] = teams[0].text.split(' // ')[0]
                    curr_match_data['Team 2'] = teams[0].text.split(' // ')[1]

        cotes = match.select(coordinates[4])
        # now we select the odds
        
        if len(cotes) >= 3:
        # again, just to prevent we select either an empty list or something irrelevant
        # do not get misled: even if the odd is empty on the site, cotes is a list of web objects,
        # so as long as the odd small rectangle if displayed len(cotes) will be >=3. 
        
            if len(cotes[0].select('span')) > 0:
            # we make the distinction of selecting the span in the selection or not selecting it
            # because sometimes the string containing the odd is inside the span...
            
                curr_match_data['Cote 1'] = cotes[0].select('span')[0].text
            else:
            # ... and sometimes it is outside the span!
            
                curr_match_data['Cote 1'] = cotes[0].text
            if len(cotes[1].select('span')) > 0:
                curr_match_data['Cote N'] = cotes[1].select('span')[0].text
            else:
                curr_match_data['Cote N'] = cotes[1].text
            if len(cotes[2].select('span')) > 0:
                curr_match_data['Cote 2'] = cotes[2].select('span')[0].text
            else:
                curr_match_data['Cote 2'] = cotes[2].text
        # NB: cote 1: odd that player 1 wins, cote N: odd that it be a draw, cote 2: player 2 wins
        
        time = match.select(coordinates[5])
        # here we select the timers (for example 17:50), which is why when there is no such object inside a box
        # it means the match is a Live event
        
        if len(time):
        # ie if we find a start time
        
            if len(time[0].select('span')):
            # sometimes the time is in a span too
            
                curr_match_data['Time'] = time[0].select('span')[0].text
            else:
                curr_match_data['Time'] = time[0].text
            # Remark: I take the first coordinate in a purely functional design. I noticed that sometimes
            # errors occured when I did not specify that I was taking the first coordinate.
            
        else:
        # if the event is live
        
            curr_match_data['Time'] = 'Live'

        curr_match_data['Site'] = coordinates[6]
        # we add a key 'site' to have some source visibility in the forecoming DataFrame
        
        data.append(curr_match_data)
        # again, data is a list of dictionaries, each dictionary represents a match

# 5. Visibility of an element to the user
- In order to open the shutter button at the end of some pages, __it is important for the button to be clickable__.
- The __problem with the is_displayed() method__ of Geckodriver is that it returns True as soon as the element is loaded in the JavaScript.
- Therefore an element can be "displayed" according the the is_displayed method, but still not clickable.
- __What I needed was a piece of code that would return True only if the element was clickable__. I do not know how to write JS, so I admit I found this convenient code on StackOverflow, which I understand though.

In [7]:
def is_element_visible(driver, element) -> bool:
    return driver.execute_script("var elem = arguments[0],                 " 
                                 "  box = elem.getBoundingClientRect(),    " 
                                 "  cx = box.left + box.width / 2,         " 
                                 "  cy = box.top + box.height / 2,         " 
                                 "  e = document.elementFromPoint(cx, cy); " 
                                 "for (; e; e = e.parentElement) {         " 
                                 "  if (e === elem)                        " 
                                 "    return true;                         " 
                                 "}                                        " 
                                 "return false;                            "
                                 , element)

# 6. Dataframes generator
- This section is dedicated to generating one dataframe for each web page scraped.
- It is hardly more than a combination of previous cells of the code.

In [8]:
Frames = []
# where we are going to stock our dataframes

for coordinates in websites_data:
# must be read: "for each web page, each one represented by a list of coordinates"
    
    browser = webdriver.Firefox()
    # let's open the driver
    
    action = ActionChains(browser)
    # we will need to perform some moving and clicking later
    
    browser.get(coordinates[0])
    # the driver gets the URL
    
    window_height = browser.execute_script("return window.innerHeight") - 150
    # window height is the height we are going to use to scroll down.
    # as you can notice we choose less than the height provided by the browser, 
    # indeed that height includes banners, so scrolling too much will have us lose data!
    
    page_height = browser.execute_script("return document.body.scrollHeight;")
    # it is the total height of the page
    
    i = 0
    # our scrolling index
    
    data = []
    # for each web page, we begin with an empty list that will contain all matches dictionaries
    
    time.sleep(2)
    # to let the JavaScript execute itself...
    
    while page_height - 20 > i * window_height:
    # as long as we have not reached the bottom of the page (with a 20 pixels margin)

        if coordinates[7] == 1:
        # dummy variable equals to one if there is a shutter button to find before we can keep scrolling
        
            L = len(browser.find_elements_by_xpath(coordinates[8]))
            # L is the length of the list of shutter buttons found
            
            if L >= 1:
            # ie if we find a button
            
                button = browser.find_element_by_xpath(coordinates[8])
                # xpath is a good way of finding the button, because it is way more precise than class name or css selector
                # and less precise than by_id, which is unique and so would require more data
                
                if is_element_visible(browser, button): 
                # if the element is clickable
                
                    action.move_to_element_with_offset(button, 5, 5)
                    action.click().perform()
                    # then let's click on it to open new scrolling space
                    
                    time.sleep(2)
                    # let's doze off for a while in order to let the driver charge the page
        
        html = browser.page_source
        # let's replace the old html by the new one, freshly collected from the new window
        
        match_data(html, data, coordinates)
        # now that everything that needed to be opened is available, it is time to implement the scraping function
        
        browser.execute_script("window.scrollBy(0, window.innerHeight - 150)")
        # scroll to the next matches
        
        page_height = browser.execute_script("return document.body.scrollHeight;")
        # refresh page height, which needs to be refreshed because there is an offset 
        
        i = i + 1
        # * window height, we have the pixels scrolled until now, which can be compared to page height

    browser.close()
    Frames.append(pd.DataFrame(data).dropna().drop_duplicates(subset=None, keep='first', inplace=False).reset_index(drop=True))
    # transforms the list of dictionaries into a Pandas dataframe, then appends that frame to Frames
    # which is a list of dataframes we are going to clean and concatenate in section 7.

In [9]:
Frames[1]
# rough version of Zebet dataframe

,Team 1,Team 2,Cote 1,Cote N,Cote 2,Time,Site
0,Union Berlin / Bayern Munich,\n ...,"\n 8,50","\n 6,00","\n 1,26",\n \n ...,Zebet
1,Bayer Leverkusen / Hoffenheim,\n ...,"\n 1,70","\n 4,20","\n 3,80",\n \n ...,Zebet
2,Augsbourg / Schalke 04,\n ...,"\n 1,88","\n 3,55","\n 3,65",\n \n ...,Zebet
3,Bayern Munich / Wolfsburg,\n ...,"\n 1,20","\n 6,75","\n 9,75",\n \n ...,Zebet
4,Werder Brême / Borussia Dortmund,\n ...,"\n 5,20","\n 4,40","\n 1,50",\n \n ...,Zebet
...,...,...,...,...,...,...,...
194,Etoile Rouge Belgrade / FK Backa,\n ...,"\n 1,01","\n 10,50","\n 32,00",\n \n ...,Zebet
195,Academico Viseu / Chaves,\n ...,"\n 3,65","\n 2,95","\n 1,82",\n \n ...,Zebet
196,FC Porto / Nacional Madeira,\n ...,"\n 1,26","\n 5,10","\n 8,00",\n \n ...,Zebet
197,Sporting CP / Sporting Farense,\n ...,"\n 1,25","\n 5,20","\n 8,50",\n \n ...,Zebet


# 7. Dataframes cleaning

In [10]:
Frames[0]['Cote 1'] = pd.to_numeric(Frames[0]['Cote 1'].str.replace(',', '.'), downcast = "float")
Frames[0]['Cote N'] = pd.to_numeric(Frames[0]['Cote N'].str.replace(',', '.'), downcast = "float")
Frames[0]['Cote 2'] = pd.to_numeric(Frames[0]['Cote 2'].str.replace(',', '.'), downcast = "float")
# the data are fine, except the odds columns, which are strings and need to be floats instead
# pandas to_numeric method works nicely, once ',' are replaced by '.'
Netbet_df = Frames[0]

team2 = [teams[1] for teams in Frames[1]['Team 1'].str.split(' / ')]
team1 = [teams[0] for teams in Frames[1]['Team 1'].str.split(' / ')]
Frames[1]['Team 1'] = team1
Frames[1]['Team 2'] = team2
# surprisingly, the output of 'Team 1' column mingles both Team 1 data and Team 2 data
# though the BS4 scraping function works just fine for all other websites...
# so here we do the split manually, and reassign the proper teams to the proper columns
# NB: the splitter here is '/' 
Frames[1]['Cote 1'] = pd.to_numeric(Frames[1]['Cote 1'].str.strip().str.replace(',', '.'), downcast = "float")
Frames[1]['Cote N'] = pd.to_numeric(Frames[1]['Cote N'].str.strip().str.replace(',', '.'), downcast = "float")
Frames[1]['Cote 2'] = pd.to_numeric(Frames[1]['Cote 2'].str.strip().str.replace(',', '.'), downcast = "float")
# as for the first frame, the problem is the type of odds data, so we change it after having made it easy
# for to_numeric method to work
Zebet_df = Frames[1]

Frames[2]['Cote 1'] = pd.to_numeric(Frames[2]['Cote 1'].str.replace(',', '.'), downcast = "float")
Frames[2]['Cote N'] = pd.to_numeric(Frames[2]['Cote N'].str.replace(',', '.'), downcast = "float")
Frames[2]['Cote 2'] = pd.to_numeric(Frames[2]['Cote 2'].str.replace(',', '.'), downcast = "float")
# same comments as above
Winamax_df = Frames[2]

France_pari_df = pd.concat([Frames[3], Frames[4], Frames[5], Frames[6], Frames[7], Frames[8]]).reset_index(drop=True)
# France pari is an exception because the data was on separate web pages, so here we concatenate the data in one frame
France_pari_df['Team 1'] = France_pari_df['Team 1'].str.strip()
France_pari_df['Team 2'] = France_pari_df['Team 2'].str.strip()
# there are blank space in teams columns, on the left and on the right of the words, so we strip them
France_pari_df['Cote 1'] = pd.to_numeric(France_pari_df['Cote 1'].str.replace(',', '.'), downcast = "float")
France_pari_df['Cote N'] = pd.to_numeric(France_pari_df['Cote N'].str.replace(',', '.'), downcast = "float")
France_pari_df['Cote 2'] = pd.to_numeric(France_pari_df['Cote 2'].str.replace(',', '.'), downcast = "float")
# again, we convert the odds columns to a nicer format

Parions_sport_df = Frames[9]
Parions_sport_df['Cote 1'] = pd.to_numeric(Parions_sport_df['Cote 1'].str.replace(',', '.'), downcast = "float")
Parions_sport_df['Cote N'] = pd.to_numeric(Parions_sport_df['Cote N'].str.replace(',', '.'), downcast = "float")
Parions_sport_df['Cote 2'] = pd.to_numeric(Parions_sport_df['Cote 2'].str.replace(',', '.'), downcast = "float")

Frames[10]['Team 1'] = Frames[10]['Team 1'].str.strip()
Frames[10]['Team 2'] = Frames[10]['Team 2'].str.strip()
Frames[10]['Cote 1'] = pd.to_numeric(Frames[10]['Cote 1'].str.replace(',', '.'), downcast = "float")
Frames[10]['Cote N'] = pd.to_numeric(Frames[10]['Cote N'].str.replace(',', '.'), downcast = "float")
Frames[10]['Cote 2'] = pd.to_numeric(Frames[10]['Cote 2'].str.replace(',', '.'), downcast = "float")
Feelingbet_df = Frames[10]

Unibet_df = Frames[11]
Unibet_df['Cote 1'] = pd.to_numeric(Unibet_df['Cote 1'].str.replace(',', '.'), downcast = "float")
Unibet_df['Cote N'] = pd.to_numeric(Unibet_df['Cote N'].str.replace(',', '.'), downcast = "float")
Unibet_df['Cote 2'] = pd.to_numeric(Unibet_df['Cote 2'].str.replace(',', '.'), downcast = "float")

Frames[12]['Team 1'] = Frames[12]['Team 1'].str.strip()
Frames[12]['Team 2'] = Frames[12]['Team 2'].str.strip()
Frames[12]['Cote 1'] = pd.to_numeric(Frames[12]['Cote 1'].str.replace(',', '.'), downcast = "float")
Frames[12]['Cote N'] = pd.to_numeric(Frames[12]['Cote N'].str.replace(',', '.'), downcast = "float")
Frames[12]['Cote 2'] = pd.to_numeric(Frames[12]['Cote 2'].str.replace(',', '.'), downcast = "float")
Betclic_df = Frames[12]

In [11]:
Winamax_df
# an example after cleaning

,Team 1,Team 2,Cote 1,Cote N,Cote 2,Time,Site
0,Marseille,Monaco,1.68,3.15,3.35,Live,Winamax
1,Newcastle,West Bromwich,2.60,1.96,3.10,Live,Winamax
2,Borussia M'Gladbach,Hertha Berlin,3.00,1.42,6.00,Live,Winamax
3,Fribourg,Arminia Bielefeld,1.05,6.25,48.00,Live,Winamax
4,Mayence,Cologne,10.00,3.20,1.29,Live,Winamax
...,...,...,...,...,...,...,...
430,Belgique,Russie,1.76,3.50,4.50,21:00,Winamax
431,Angleterre,Croatie,1.82,3.35,4.40,15:00,Winamax
432,Pays-Bas,Ukraine,1.56,3.85,5.60,21:00,Winamax
433,Espagne,Suède,1.38,4.50,7.50,21:00,Winamax


# 8. Dataframes concatenation

In [12]:
Frame = pd.concat([Netbet_df, Betclic_df, Winamax_df, France_pari_df, Parions_sport_df, Feelingbet_df, Unibet_df, Zebet_df])
Frame = Frame.reset_index(drop = True)
# we concatenate all frames in one big dataframe

Frame['Match id'] = [n for n in range(len(Frame['Team 1']))]
# we define a new column 'Match id' as the index. It will be modified in the section below

Frame['Mixed matches'] = Frame['Team 1'] + Frame['Team 2']
Frame['Mixed matches'] = Frame['Mixed matches'].str.lower().str.replace(' ', '')
# above the goal is to convert contestants names into the most basic string possible,
# in order to make it easier for two match titles to be compared

Frame['Time'] = Frame['Time'].str.replace(' ', '').str.strip()
# we erase blank space from Time column, all rows at once.

In [13]:
Frame
# we now have one big dataframe which combines all data. Next section is about how to prepare it for later clustering

,Team 1,Team 2,Cote 1,Cote N,Cote 2,Time,Site,Match id,Mixed matches
0,NK Varazdin,Hajduk Split,4.70,3.30,1.35,4min,Netbet,0,nkvarazdinhajduksplit
1,Olimpia Ljubljana,Tabor Sezana,1.37,3.30,4.50,14min,Netbet,1,olimpialjubljanataborsezana
2,Gornik Zabrze,Cracovia Krakow,1.90,2.95,3.05,29min,Netbet,2,gornikzabrzecracoviakrakow
3,Olympic Safi,WAC Casablanca,3.95,2.67,1.61,29min,Netbet,3,olympicsafiwaccasablanca
4,Connah'S Quay,The New Saints,2.97,3.05,1.71,29min,Netbet,4,connah'squaythenewsaints
...,...,...,...,...,...,...,...,...,...
2333,Etoile Rouge Belgrade,FK Backa,1.01,10.50,32.00,13/1217:00,Zebet,2333,etoilerougebelgradefkbacka
2334,Academico Viseu,Chaves,3.65,2.95,1.82,19/1216:00,Zebet,2334,academicoviseuchaves
2335,FC Porto,Nacional Madeira,1.26,5.10,8.00,20/1221:00,Zebet,2335,fcportonacionalmadeira
2336,Sporting CP,Sporting Farense,1.25,5.20,8.50,19/1221:30,Zebet,2336,sportingcpsportingfarense


# 9. Clustering by orthographic similarity
- The goal of this section is to attribute one same 'Match id' to all identical matches on different sites.
- The issue is to find the adapted ratio of similarity in the 'Mixed matches' column to have neither falsely positive results nor falsely negative rows.

In [14]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
# SequenceMatcher computes a similarity ratio between two strings of characters

def next_rows_sorter(k, frame, p, i):
# compares "Mixed matches" of row k and row i > k, if i is not beyond height of the dataframe.
    if i == len(frame):
        return
    #if frame['Match id'][i] < i:
    # if row i has already been clusterized
     #   return
    if similar(frame['Mixed matches'][k], frame['Mixed matches'][i]) >= p:
            frame['Match id'][i] = frame['Match id'][k]
            # if similar enough in terms of "Mixed matches" strings, "Match id" of row i is set equal to the one of row k
            # by doing so, clusters are created and their keys are in the "Match id" column
    return next_rows_sorter(k, frame, p, i+1)
    # the afore-explained process is implemented on all following rows

def sorter(frame, p, k = 0):
# implements next_rows_sorter on all rows except the last one
    if k == len(frame) - 1:
        return
    if frame['Match id'][k] < k:
        return
    next_rows_sorter(k, frame, p, k+1)
    return sorter(frame, p, k+1)

p = int(input('Which similarity percentage do you wish to choose? NB: 70 is low and will allow misleading results; above 90 may be too high and drop conclusive rows, so you may want to take p between 80 and 90.'))
# percentage of 85% is satisfactory

sorter(Frame, p/100)
# implements sorter

Which similarity percentage do you wish to choose? NB: 70 is low and will allow misleading results; above 90 may be too high and drop conclusive rows, so you may want to take p between 80 and 90.85


<ipython-input-14-4ae37128cad5>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame['Match id'][i] = frame['Match id'][k]


In [15]:
Frame.tail(50)
# as you can see below, only few matches have not been clusterized (I am talking about the 'Match id' column)

,Team 1,Team 2,Cote 1,Cote N,Cote 2,Time,Site,Match id,Mixed matches
2288,Estoril,Boavista Porto,2.42,2.82,2.55,12/1222:00,Zebet,43,estorilboavistaporto
2289,CD Cova Piedade,Moreirense,3.35,3.10,1.83,13/1212:00,Zebet,60,cdcovapiedademoreirense
2290,Espagne,Suède,1.33,4.60,7.75,14/0621:00,Zebet,2290,espagnesuède
2291,Internacional RS,Botafogo RJ,1.48,3.40,5.20,12/1223:00,Zebet,46,internacionalrsbotafogorj
2292,Palmeiras,Bahia BA,1.45,3.55,5.30,12/1223:00,Zebet,47,palmeirasbahiaba
2293,Atletico PR,Atletico MG,3.30,3.25,1.83,12/1221:00,Zebet,37,atleticopratleticomg
2294,Bragantino SP,Fortaleza CE,1.65,3.15,4.30,12/1221:00,Zebet,36,bragantinospfortalezace
2295,Goias GO,Gremio,5.40,3.55,1.45,13/1201:00,Zebet,52,goiasgogremio
2296,Ceara CE,Atletico GO,1.85,2.92,3.65,13/1201:00,Zebet,51,cearaceatleticogo
2297,Corinthians,Sao Paulo,3.55,3.05,1.83,13/1222:15,Zebet,160,corinthianssaopaulo


# 10. Surebet finder (or not)
- In this section let's conclude by maximizing the odds for every 'Match id' cluster, ie for each match
- We check if the condition TRJ > 1 is satisfied, which was our primal goal, then display the results.

In [16]:
def TRJ(c1, c2, c3):
# see notes attached to the code for full explanation regarding TRJ
    if c1 != 0 and c2 != 0 and c3 != 0:
        return 1/(1/c1 + 1/c2 + 1/c3)
    else:
        return 0

ids = []
# to ids we will append the value -1 for matches whose TRJ are <= 1, otherwise the "Match id" that corresponds to the match

for id in Frame['Match id']:
    trj = TRJ(max(Frame[Frame['Match id'] == id]['Cote 1']), max(Frame[Frame['Match id'] == id]['Cote N']), max(Frame[Frame['Match id'] == id]['Cote 2']))
    # trj is the computation of the TRJ, each odd being the maximum odd for all odds (of different sites)
    # of the same result of the match (we select each specific match with Frame['Match id'] == id, which means "let's 
    # consider one match at a time")
    
    if trj > 1:
        ids.append(id)
    else:
        ids.append(-1)
    # explained above
    
Results_df = Frame[Frame['Match id'] == ids].sort_values(by = ['Match id'])
# Results_df is a dataframe with all positive results 

results_count = len(list(dict.fromkeys(Results_df['Match id'])))
# we count all distinct values of 'Match id', ie all different matches with a TRJ > 1

if results_count == 0:
    print('With ', p, '% of similarity, no match that could lead to a surebet was found.')
if results_count > 0:
    print('With ', p, '% of similarity, ', results_count, ' matches were found and could lead to a surebet. See Results_df with the following "Match id" values for full understanding: ', list(dict.fromkeys(ids)))
    # display of the results

With  85 % of similarity, no match that could lead to a surebet was found.


# *Conclusion and remarks* 

- __Failures__ 

    - I spent a lot of time trying to unsuccesfully *__scrap Bwin__*. The website is really designed to make it difficult to scroll, and down arrows to scroll down are either hidden or unclickable. 
    - Initially, I wanted to *__scrap all dates__* of football events, and then link them to their respective match box. Indeed dates are the only data out of each match box selected with BS4 tool "soup.select". Because BS4 reads html page source code, there is no way to point out the locus of the web element on the screen directly with BS4. As a consequence BeautifulSoup dates selection had to be converted into Selenium webdriver elements. Using a javascript function made it possible, yet very heavy when running the algorithm. I ditched the idea, since it was not useful enough to stay in the final script.
<br>
<br>
- __Remarks__

    - At first __I wanted to do a predictive program__, but when it comes to online betting there is no way to predict results except in an historical manner with regressions. It is what bookmakers do, and they do it better given the knowledge, experience of modelling and exclusive data they have.
    - When beginning the code, __I knew there would not be arbitrage opportunities__ at the end of the day, given many hedgefunds have specialized in online betting speculation, and would clearly have come up with such a simple idea if it was a situation that could actually happen. However, to me __it was a nice project__, because it combined a lot of coding technics seen in class, and went further in terms of remote scraping.